In [1]:
import pandas as pd

# Load datasets
path_prefix = "C:\\Users\\gbemi\\Desktop\\Capstone\\"
delay_df = pd.read_csv(path_prefix + "Delay_df.csv")
weather_events = pd.read_csv(path_prefix + "Weather_events.csv")

In [2]:
delay_df.columns

Index(['FL_DATE', 'OP_CARRIER', 'OP_CARRIER_FL_NUM', 'ORIGIN', 'DEST',
       'CRS_DEP_TIME', 'DEP_TIME', 'DEP_DELAY', 'TAXI_OUT', 'WHEELS_OFF',
       'WHEELS_ON', 'TAXI_IN', 'CRS_ARR_TIME', 'ARR_TIME', 'ARR_DELAY',
       'CANCELLED', 'CANCELLATION_CODE', 'DIVERTED', 'CRS_ELAPSED_TIME',
       'ACTUAL_ELAPSED_TIME', 'AIR_TIME', 'DISTANCE', 'CARRIER_DELAY',
       'WEATHER_DELAY', 'NAS_DELAY', 'SECURITY_DELAY', 'LATE_AIRCRAFT_DELAY'],
      dtype='object')

In [4]:
weather_events.columns


Index(['EventId', 'Type', 'Severity', 'StartTime(UTC)', 'EndTime(UTC)',
       'Precipitation(in)', 'TimeZone', 'AirportCode', 'LocationLat',
       'LocationLng', 'City', 'County', 'State', 'ZipCode'],
      dtype='object')

In [13]:
import pandas as pd
import pytz

# Assuming you've already loaded your datasets into two DataFrames:
# delay_df and weather_events_df

# Convert CRS_DEP_TIME and CRS_ARR_TIME to a datetime format:
delay_df['CRS_DEP_TIME'] = delay_df['CRS_DEP_TIME'].apply(lambda x: f"{int(x)//100:02d}:{int(x)%100:02d}")
delay_df['CRS_ARR_TIME'] = delay_df['CRS_ARR_TIME'].apply(lambda x: f"{int(x)//100:02d}:{int(x)%100:02d}")
delay_df['DEP_DATETIME'] = pd.to_datetime(delay_df['FL_DATE'] + ' ' + delay_df['CRS_DEP_TIME'])
delay_df['ARR_DATETIME'] = pd.to_datetime(delay_df['FL_DATE'] + ' ' + delay_df['CRS_ARR_TIME'])



ValueError: invalid literal for int() with base 10: '08:56'

In [ ]:
# Adjust the times to UTC:
merged_df = delay_df.merge(weather_events_df[['AirportCode', 'TimeZone']], left_on='ORIGIN', right_on='AirportCode', how='left')
merged_df.rename(columns={'TimeZone': 'ORIGIN_TimeZone'}, inplace=True)
merged_df = merged_df.merge(weather_events_df[['AirportCode', 'TimeZone']], left_on='DEST', right_on='AirportCode', how='left', suffixes=('', '_DEST'))

def adjust_to_utc(row, column, timezone_column):
    local = pytz.timezone(row[timezone_column])
    local_dt = local.localize(row[column], is_dst=None)
    utc_dt = local_dt.astimezone(pytz.utc)
    return utc_dt

merged_df['DEP_DATETIME_UTC'] = merged_df.apply(lambda row: adjust_to_utc(row, 'DEP_DATETIME', 'ORIGIN_TimeZone'), axis=1)
merged_df['ARR_DATETIME_UTC'] = merged_df.apply(lambda row: adjust_to_utc(row, 'ARR_DATETIME', 'ORIGIN_TimeZone'), axis=1)



In [ ]:
# Weather Events at Origin and Destination:
origin_affected = merged_df[(merged_df['DEP_DATETIME_UTC'] >= merged_df['StartTime(UTC)']) & 
                            (merged_df['DEP_DATETIME_UTC'] <= merged_df['EndTime(UTC)'])]
dest_affected = merged_df[(merged_df['ARR_DATETIME_UTC'] >= merged_df['StartTime(UTC)']) & 
                          (merged_df['ARR_DATETIME_UTC'] <= merged_df['EndTime(UTC)'])]


In [ ]:

# Combining the Results:
combined_affected = pd.concat([origin_affected, dest_affected])
combined_affected = combined_affected.drop_duplicates()




In [ ]:
# Further Analysis:
mean_delay = combined_affected['DEP_DELAY'].mean()
total_cancellations = combined_affected['CANCELLED'].sum()

print(f"Mean delay due to weather events: {mean_delay} minutes")
print(f"Total cancellations due to weather events: {total_cancellations}")